In [ ]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [ ]:
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')

    if not api_key:
        raise ValueError(" GEMINI_API_KEY not found in Colab userdata. Please set it before running.")
except Exception as e:
    print(f"Error while retrieving API key: {e}")
    api_key = No

In [ ]:
try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    from IPython.display import display, Markdown
    import textwrap
except ImportError as e:
    raise ImportError("Required libraries not found. Please reinstall using !pip install -U langchain-google-genai") from e


In [ ]:
try:
    if api_key:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            temperature=0.2,
            google_api_key=api_key
        )
    else:
        llm = None
        print(" Model not initialized due to missing API key.")
except Exception as e:
    print(f"Error initializing the LLM: {e}")
    llm = No

In [ ]:
def to_markdown(text):
    try:
        text = text.replace('•', '  *')
        return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
    except Exception as e:
        print(f"Error in to_markdown: {e}")
        return Markdown(">  Unable to render markdown properly.")


In [ ]:

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
   temperature=0.2,
    google_api_key=api_key
)


In [ ]:
db_schema = """
You are an expert SQL query generator and database analyst.
The database schema is as follows:

TABLE: Customers(id, name, email, city)
TABLE: Orders(id, customer_id, product, amount, order_date)

Instructions:
* Always generate **syntactically correct**, **optimized**, and **executable** SQL queries.
* Use **JOINs**, **GROUP BY**, **aggregations**, and **subqueries** when necessary.
* Consider relationships between tables (e.g., Orders.customer_id = Customers.id).
* If the question involves totals, averages, or counts — use appropriate **aggregate functions**.
* Use **clear aliases** for readability (e.g., c for Customers, o for Orders).
* Never include explanations or comments in the final SQL output — only the query itself.
* Prefer ANSI SQL syntax to ensure compatibility across databases.

Now generate the correct and optimized SQL query for the given question.
"""


In [ ]:
def text_to_sql_markdown(user_question):
    if not llm:
        print(" LLM model not initialized. Please check your API key or initialization.")
        return

    try:
        prompt = f"{db_schema}\n\nQuestion: {user_question}\n\nSQL Query:"
        response = llm.invoke(prompt)

        if not hasattr(response, "content") or not response.content.strip():
            raise ValueError("Empty response received from model.")

        sql_query = response.content.strip()

        display(to_markdown(f"**Question:** {user_question}\n\n**SQL Query:**\n```sql\n{sql_query}\n```"))

    except Exception as e:
        print(f" Error generating SQL for '{user_question}': {e}")
        display(Markdown(f">  Could not generate SQL for this question: **{user_question}**"))


In [ ]:
text_to_sql_markdown("List all customers from New York")
text_to_sql_markdown("Show total order amount for each customer")
text_to_sql_markdown("Find customers who have placed more than 3 orders")

> **Question:** List all customers from New York
> 
> **SQL Query:**
> ```sql
> ```sql
> SELECT
>   c.id,
>   c.name,
>   c.email,
>   c.city
> FROM Customers AS c
> WHERE
>   c.city = 'New York';
> ```
> ```

> **Question:** Show total order amount for each customer
> 
> **SQL Query:**
> ```sql
> ```sql
> SELECT
>   c.name,
>   SUM(o.amount) AS total_order_amount
> FROM Customers AS c
> JOIN Orders AS o
>   ON c.id = o.customer_id
> GROUP BY
>   c.id,
>   c.name
> ORDER BY
>   c.name;
> ```
> ```

> **Question:** Find customers who have placed more than 3 orders
> 
> **SQL Query:**
> ```sql
> ```sql
> SELECT
>   c.id,
>   c.name
> FROM Customers AS c
> JOIN Orders AS o
>   ON c.id = o.customer_id
> GROUP BY
>   c.id,
>   c.name
> HAVING
>   COUNT(o.id) > 3;
> ```
> ```